In [1]:
#Importing Required Libraries

import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve

In [3]:
# Reading in the data
retail_data = pd.read_excel('Data.xlsx')

#Getting a sense of what the data is like

retail_data.sample(7)

,SalesOrderNumber,ProductID,ProductName,Quantity,OrderDate,UnitPrice,CustomerKey,Gender,MaritalStatus,EnglishEducation,EnglishOccupation,Country
1471,SO45679,314,"Road-150 Red, 56",1,2011-09-13,3578.27,11540,M,S,Graduate Degree,Skilled Manual,United States
8936,SO52739,528,Mountain Tire Tube,1,2013-02-14,4.99,19726,M,M,Graduate Degree,Management,Canada
27784,SO61035,355,"Mountain-200 Silver, 42",1,2013-06-28,2319.99,14181,F,M,Bachelors,Management,Germany
34593,SO64041,374,"Road-250 Black, 44",1,2013-08-11,2443.35,13529,F,S,Partial College,Manual,Australia
14670,SO55383,574,"Touring-1000 Blue, 50",1,2013-03-30,2384.07,15357,M,M,Partial High School,Professional,Germany
32685,SO63107,217,"Sport-100 Helmet, Black",1,2013-07-30,34.99,25133,F,S,Bachelors,Professional,Australia
51186,SO71181,217,"Sport-100 Helmet, Black",1,2013-11-20,34.99,11569,M,M,Bachelors,Clerical,United Kingdom


In [4]:
# Checking for missing entries in the dataset
retail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60398 entries, 0 to 60397
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   SalesOrderNumber   60398 non-null  object        
 1   ProductID          60398 non-null  int64         
 2   ProductName        60398 non-null  object        
 3   Quantity           60398 non-null  int64         
 4   OrderDate          60398 non-null  datetime64[ns]
 5   UnitPrice          60398 non-null  float64       
 6   CustomerKey        60398 non-null  int64         
 7   Gender             60398 non-null  object        
 8   MaritalStatus      60398 non-null  object        
 9   EnglishEducation   60398 non-null  object        
 10  EnglishOccupation  60398 non-null  object        
 11  Country            60398 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(7)
memory usage: 5.5+ MB


In [5]:
#table to keep track of each itemID and its description
item_lookup = retail_data[['ProductID', 'ProductName']].drop_duplicates()# Only get unique items and description pairs
item_lookup['ProductID']= item_lookup.ProductID.astype(str)#Changing to string for ease of future lookup

In [6]:
item_lookup.head() # to see what makes up the dataset

,ProductID,ProductName
0,310,"Road-150 Red, 62"
1,346,"Mountain-100 Silver, 44"
3,336,"Road-650 Black, 62"
5,311,"Road-150 Red, 44"
7,351,"Mountain-100 Black, 48"


In [7]:
item_lookup.info() # to check for empty cells.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 7276
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ProductID    158 non-null    object
 1   ProductName  158 non-null    object
dtypes: object(2)
memory usage: 3.7+ KB


In [8]:
# Convert Customer ID to int

retail_data['CustomerKey']= retail_data.CustomerKey.astype(int)

In [9]:
#Making 'ProductID the index'
retail_data.set_index('ProductID', inplace=True)

In [10]:
retail_data.head(2)

,SalesOrderNumber,ProductName,Quantity,OrderDate,UnitPrice,CustomerKey,Gender,MaritalStatus,EnglishEducation,EnglishOccupation,Country
ProductID,,,,,,,,,,,
310,SO43697,"Road-150 Red, 62",1,2010-12-29,3578.27,21768,M,S,Bachelors,Management,Canada
346,SO43698,"Mountain-100 Silver, 44",1,2010-12-29,3399.99,28389,F,S,High School,Manual,France


In [11]:
#resetting the index
retail_data.reset_index(inplace=True)

In [12]:
#Selecting relevant columns#

retail_data = retail_data[['ProductID', 'Quantity', 'CustomerKey']]

In [13]:
#Grouping records by customer key and product ID

group_retail = retail_data.groupby(['CustomerKey','ProductID']).sum().reset_index() #group items together 

In [89]:
# Checking the dataset

group_retail.head(5)

,CustomerKey,ProductID,Quantity
0,11000,214,1
1,11000,344,1
2,11000,353,1
3,11000,485,1
4,11000,488,1


In [15]:
# replace zero purchase with one to indicate purchase and take care of situations where products were returned.

#group_retail.Quantity.loc[group_retail.Quantity == 0] = 1 

In [30]:
#Show customers that purchased at least 1 item.
grouped_purchased=group_retail.query('Quantity>0') 

In [31]:
grouped_purchased.tail(10)

,CustomerKey,ProductID,Quantity
59041,29478,575,1
59042,29479,358,1
59043,29480,217,1
59044,29480,225,1
59045,29480,477,1
59046,29480,479,1
59047,29480,562,1
59048,29481,349,1
59049,29482,358,1
59050,29483,360,1


In [34]:
grouped_purchased.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59051 entries, 0 to 59050
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   CustomerKey  59051 non-null  int64
 1   ProductID    59051 non-null  int64
 2   Quantity     59051 non-null  int64
dtypes: int64(3)
memory usage: 1.8 MB


In [35]:
# To get unique customers
customers = list(np.sort(grouped_purchased.CustomerKey.unique()))


In [36]:
#To get unique products
products = list(grouped_purchased.ProductID.unique())


In [37]:
#For all the purchases made
quantity= list(grouped_purchased.Quantity)



In [46]:
#Getting the associated row indices
rows= grouped_purchased.CustomerKey.astype('category').cat.codes

In [47]:
#Getting the associated column indices
cols = grouped_purchased.ProductID.astype('category').cat.codes

In [48]:
purchase_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape = (len(customers), len(products)))

In [49]:
purchase_sparse # We have 18484 customers with 158 items

<18484x158 sparse matrix of type '<class 'numpy.intc'>'
	with 59051 stored elements in Compressed Sparse Row format>

In [50]:
# For collaborative filtering to work, spacity has to be 
# less than 99.5%.

# To determine the sparsity, number of possible interactions in the matrix

matrix_size = purchase_sparse.shape[0]* purchase_sparse.shape[1]

In [51]:
#Number of items interacted with
num_purchases = len(purchase_sparse.nonzero()[0])

In [52]:
sparsity = 100*(1-(num_purchases/matrix_size))
sparsity #Maximum allowable spacity is 99.5

97.97803231806365

## Creating a Training and Validation Set

In [72]:
import random

In [73]:
def make_train(purchase_sparse, pct_test = 0.4):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = purchase_sparse.copy() # Make a copy of the original set to act as test set
    test_set[test_set != 0] =1 # Store the test set as a binary preference matrix
    training_set = purchase_sparse.copy() # Make a copy of the original set we can alter to act as training set
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

In [74]:
product_train, product_test, product_users_altered = make_train(purchase_sparse, pct_test = 0.4)
# train and test set successfully split.

## Implementing ALS for implicit feedback

In [75]:
def implicit_weighted_ALS(training_set, lambda_val = 0.1, alpha = 40, iterations = 10, rank_size = 20, seed = 0):
    '''
    Implicit weighted ALS taken from Hu, Koren, and Volinsky 2008. Designed for alternating least squares and implicit
    feedback based collaborative filtering. 
    
    parameters:
    
    training_set - Our matrix of ratings with shape m x n, where m is the number of users and n is the number of items.
    Should be a sparse csr matrix to save space. 
    
    lambda_val - Used for regularization during alternating least squares. Increasing this value may increase bias
    but decrease variance. Default is 0.1. 
    
    alpha - The parameter associated with the confidence matrix discussed in the paper, where Cui = 1 + alpha*Rui. 
    The paper found a default of 40 most effective. Decreasing this will decrease the variability in confidence between
    various ratings.
    
    iterations - The number of times to alternate between both user feature vector and item feature vector in
    alternating least squares. More iterations will allow better convergence at the cost of increased computation. 
    The authors found 10 iterations was sufficient, but more may be required to converge. 
    
    rank_size - The number of latent features in the user/item feature vectors. The paper recommends varying this 
    between 20-200. Increasing the number of features may overfit but could reduce bias. 
    
    seed - Set the seed for reproducible results
    
    returns:
    
    The feature vectors for users and items. The dot product of these feature vectors should give you the expected 
    "rating" at each point in your original matrix. 
    '''
    
    # first set up our confidence matrix
    
    conf = (alpha*training_set) # To allow the matrix to stay sparse, I will add one later when each row is taken 
                                # and converted to dense. 
    num_user = conf.shape[0]
    num_item = conf.shape[1] # Get the size of our original ratings matrix, m x n
    
    # initialize our X/Y feature vectors randomly with a set seed
    rstate = np.random.RandomState(seed)
    
    X = sparse.csr_matrix(rstate.normal(size = (num_user, rank_size))) # Random numbers in a m x rank shape
    Y = sparse.csr_matrix(rstate.normal(size = (num_item, rank_size))) # Normally this would be rank x n but we can 
                                                                 # transpose at the end. Makes calculation more simple.
    X_eye = sparse.eye(num_user)
    Y_eye = sparse.eye(num_item)
    lambda_eye = lambda_val * sparse.eye(rank_size) # Our regularization term lambda*I. 
    
    # We can compute this before iteration starts. 
    
    # Begin iterations
   
    for iter_step in range(iterations): # Iterate back and forth between solving X given fixed Y and vice versa
        # Compute yTy and xTx at beginning of each iteration to save computing time
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)
        # Being iteration to solve for X based on fixed Y
        for u in range(num_user):
            conf_samp = conf[u,:].toarray() # Grab user row from confidence matrix and convert to dense
            pref = conf_samp.copy() 
            pref[pref != 0] = 1 # Create binarized preference vector 
            CuI = sparse.diags(conf_samp, [0]) # Get Cu - I term, don't need to subtract 1 since we never added it 
            yTCuIY = Y.T.dot(CuI).dot(Y) # This is the yT(Cu-I)Y term 
            yTCupu = Y.T.dot(CuI + Y_eye).dot(pref.T) # This is the yTCuPu term, where we add the eye back in
                                                      # Cu - I + I = Cu
            X[u] = spsolve(yTy + yTCuIY + lambda_eye, yTCupu) 
            # Solve for Xu = ((yTy + yT(Cu-I)Y + lambda*I)^-1)yTCuPu, equation 4 from the paper  
        # Begin iteration to solve for Y based on fixed X 
        for i in range(num_item):
            conf_samp = conf[:,i].T.toarray() # transpose to get it in row format and convert to dense
            pref = conf_samp.copy()
            pref[pref != 0] = 1 # Create binarized preference vector
            CiI = sparse.diags(conf_samp, [0]) # Get Ci - I term, don't need to subtract 1 since we never added it
            xTCiIX = X.T.dot(CiI).dot(X) # This is the xT(Cu-I)X term
            xTCiPi = X.T.dot(CiI + X_eye).dot(pref.T) # This is the xTCiPi term
            Y[i] = spsolve(xTx + xTCiIX + lambda_eye, xTCiPi)
            # Solve for Yi = ((xTx + xT(Cu-I)X) + lambda*I)^-1)xTCiPi, equation 5 from the paper
    # End iterations
    return X, Y.T # Transpose at the end to make up for not being transposed at the beginning. 
                         # Y needs to be rank x n. Keep these as separate matrices for scale reasons. 

In [76]:
user_vecs, item_vecs = implicit_weighted_ALS(product_train, lambda_val = 0.1, alpha = 40, iterations = 10,
                                            rank_size = 20)

In [77]:
#for first user
user_vecs[0,:].dot(item_vecs).toarray()[0,:5]

array([ 0.95898136,  0.15293903,  0.2703755 , -0.32327657,  0.28164163])

## Speeding up ALS

In [78]:
import implicit

In [79]:
alpha =40
user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'), 
                                                          factors=1, 
                                                          regularization = 0.3, 
                                                         iterations = 50)

This method is deprecated. Please use the AlternatingLeastSquares class instead


In [80]:
# Evaluating the Recommender Systems
from sklearn import metrics

In [81]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

In [82]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [83]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

(0.783, 0.833)

In [84]:
# To confirm if our recommendation system is predicting correctly
customers_arr = np.array(customers) # Array of customers IDs from the ratings matrix
products_arr = np.array(products) # Array f product IDs from the ratings matrix

In [85]:
def get_items_purchased(CustomerKey, mf_train, customers_list, ProductName, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    CustomerKey - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    ProductName - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == CustomerKey)[0][0] # Returns the index row of our CustomerKey
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = ProductName[purchased_ind] # Get the ProductID for our purchased items
    return item_lookup.loc[item_lookup.ProductID.isin(prod_codes)]

In [86]:
customers_arr[ :5]

array([11000, 11001, 11002, 11003, 11004], dtype=int64)

In [87]:
#To get items the customer purchased

get_items_purchased(11000, product_train, customers_arr, products_arr, item_lookup)

,ProductID,ProductName
